In [1]:
#import libraries
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime

In [2]:
def log_progress(message):
    '''This function logs the mentioned message of a given stage of the code execution to a log file.'''
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file,"a") as f: 
        f.write(timestamp + ',' + message + '\n') 

In [3]:
def extract(url):
    '''This function aims to extract the required information from the website and save it to a data frame.
        url - the url link of the website
    '''
    #create a dataframe
    df = pd.DataFrame(columns=["Name", "MC_USD_Billion"])
    
    #load the webpage for webscraping
    html_page = requests.get(url).text
    soup = BeautifulSoup(html_page, 'html.parser')
    
    #extraction of information from the webpage
    for row in soup.find_all("tbody")[1].find_all("tr"):
        cols = row.find_all("td")
        #check for columns
        if len(cols) == 0:
            continue
        else:
            #adds data respectively to the columns
            df = pd.concat([df, pd.DataFrame([{"Name":cols[1].text.strip(), "MC_USD_Billion":cols[2].text.strip()}])], ignore_index=True)
            
    return df

In [4]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
extract(url)

,Name,MC_USD_Billion
0,Industrial and Commercial Bank of China Limited,"5,742.86"
1,China Construction Bank,"5,016.81"
2,Agricultural Bank of China,"4,919.03"
3,Bank of China,"4,192.12"
4,JPMorgan Chase,"3,868.24"
...,...,...
95,National Bank of Canada,312.67
96,Macquarie Group,313.47
97,Nationwide Building Society,311.93
98,Raiffeisen Gruppe,303.50


In [5]:
def transform(df, csv_file):
    '''This function uses the extracted data and a CSV file for exchange rate information.
        - Add 3 columns to the data frame 
        - New columns containing the transformed market cap respectively using the exchange rate information from the CSV file
    '''
    #convert object to float type for the usd billion column
    df["MC_USD_Billion"] = df["MC_USD_Billion"].str.replace(',','') #remove commas
    df["MC_USD_Billion"] = df["MC_USD_Billion"].astype(float) #convert column to float dtype
    
    #read exchange rate csv file and put it into a dictionary
    df2 = pd.read_csv(csv_file)
    rates = df2.set_index("Currency").to_dict()["Rate"]
    
    #add additional columns for gbp, inr and eur with corresponding data from the usd column
    df["MC_GBP_Billion"] = [np.round(x*rates["GBP"],2) for x in df["MC_USD_Billion"]]
    df["MC_EUR_Billion"] = [np.round(x*rates["EUR"],2) for x in df["MC_USD_Billion"]]
    df["MC_INR_Billion"] = [np.round(x*rates["INR"],2) for x in df["MC_USD_Billion"]]
    
    return df

In [6]:
exchange_rate = 'exchange_rate.csv'
transform(extract(url), exchange_rate)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,Industrial and Commercial Bank of China Limited,5742.86,4594.29,5340.86,476370.24
1,China Construction Bank,5016.81,4013.45,4665.63,416144.39
2,Agricultural Bank of China,4919.03,3935.22,4574.70,408033.54
3,Bank of China,4192.12,3353.70,3898.67,347736.35
4,JPMorgan Chase,3868.24,3094.59,3597.46,320870.51
...,...,...,...,...,...
95,National Bank of Canada,312.67,250.14,290.78,25935.98
96,Macquarie Group,313.47,250.78,291.53,26002.34
97,Nationwide Building Society,311.93,249.54,290.09,25874.59
98,Raiffeisen Gruppe,303.50,242.80,282.26,25175.32


In [7]:
def load_to_csv(df, final_csv_path):
    '''This function saves the final dataframe as a CSV file'''
    df.to_csv(final_csv_path)
    
def load_to_db(df, sql_connection, table_name):
    '''This function saves the final dataframe as a database table'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [8]:
def run_query(query_statement, sql_connection):
    '''This function run sql queries'''
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_statement)
    print(query_output)

In [9]:
#initialize known entities
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
exchange_rate = 'exchange_rate.csv'
db_name = "Banks.db"
final_csv = "Largest_banks.csv"
table_name = "Largest_banks"
log_file = 'code_log.txt'
conn = sqlite3.connect(db_name) #create and connect to a database

In [10]:
# Log the initialization of the ETL process 
log_progress("ETL Started") 
 
# Log the beginning of the Extraction process 
log_progress("Extract phase Started") 
extracted_data = extract(url) 
print('Extracted Data')
print(extracted_data)
 
# Log the completion of the Extraction process 
log_progress("Extract phase Ended") 
 
# Log the beginning of the Transformation process 
log_progress("Transform phase Started") 
transformed_data = transform(extracted_data,exchange_rate) 
print("Transformed Data") 
print(transformed_data) 
 
# Log the completion of the Transformation process 
log_progress("Transform phase Ended") 
 
# Log the beginning of the Loading process 
log_progress("Load phase Started") 
load_to_csv(transformed_data,final_csv)
load_to_db(transformed_data, conn, table_name)

# Log the completion of the Loading process 
log_progress("Load phase Ended")
 
# Log the completion of the ETL process 
log_progress("ETL Job Ended") 



Extracted Data
                                               Name MC_USD_Billion
0   Industrial and Commercial Bank of China Limited       5,742.86
1                           China Construction Bank       5,016.81
2                        Agricultural Bank of China       4,919.03
3                                     Bank of China       4,192.12
4                                    JPMorgan Chase       3,868.24
..                                              ...            ...
95                          National Bank of Canada         312.67
96                                  Macquarie Group         313.47
97                      Nationwide Building Society         311.93
98                                Raiffeisen Gruppe         303.50
99                             First Abu Dhabi Bank         302.22

[100 rows x 2 columns]
Transformed Data
                                               Name  MC_USD_Billion  \
0   Industrial and Commercial Bank of China Limited         5742.86  

In [11]:
#query 1: Select entire table
query_1 = f"SELECT * FROM {table_name}"
run_query(query_1, conn)

SELECT * FROM Largest_banks
                                               Name  MC_USD_Billion  \
0   Industrial and Commercial Bank of China Limited         5742.86   
1                           China Construction Bank         5016.81   
2                        Agricultural Bank of China         4919.03   
3                                     Bank of China         4192.12   
4                                    JPMorgan Chase         3868.24   
..                                              ...             ...   
95                          National Bank of Canada          312.67   
96                                  Macquarie Group          313.47   
97                      Nationwide Building Society          311.93   
98                                Raiffeisen Gruppe          303.50   
99                             First Abu Dhabi Bank          302.22   

    MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          4594.29         5340.86       476370.24  
1          40

In [12]:
#query 2: Select name of the top 5 banks
query_2 = f"SELECT Name FROM {table_name} Limit 5"
run_query(query_2, conn)

SELECT Name FROM Largest_banks Limit 5
                                              Name
0  Industrial and Commercial Bank of China Limited
1                          China Construction Bank
2                       Agricultural Bank of China
3                                    Bank of China
4                                   JPMorgan Chase


In [13]:
#query 3: Average Market Cap (GBP)
query_3 = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
run_query(query_3, conn)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0             896.9811


In [14]:
#close connection to database
conn.close()